In [1]:
# !pip3 install -U altair

In [2]:
import altair as alt

# Data for the graph
nodes = [
    {"id": "Clássica", "group": "Física"},
    {"id": "Quântica", "group": "Física"},
    {"id": "Cálculo Diferencial e Integral", "group": "Matemática"},
    {"id": "Álgebra Linear", "group": "Matemática"},
    {"id": "Estatística e Probabilidade", "group": "Matemática"},
    {"id": "Geometria Diferencial", "group": "Matemática"},
    {"id": "Teoria dos Grafos", "group": "Matemática"},
    {"id": "Monte Carlo", "group": "Métodos"},
    {"id": "Dinâmica Molecular", "group": "Métodos"},
    {"id": "Integração Numérica", "group": "Métodos"},
]

edges = [
    {"source": "Clássica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Quântica", "target": "Cálculo Diferencial e Integral"},
    {"source": "Cálculo Diferencial e Integral", "target": "Integração Numérica"},
    {"source": "Integração Numérica", "target": "Dinâmica Molecular"},
    {"source": "Álgebra Linear", "target": "Dinâmica Molecular"},
    {"source": "Estatística e Probabilidade", "target": "Monte Carlo"},
    {"source": "Estatística e Probabilidade", "target": "Dinâmica Molecular"},
    {"source": "Geometria Diferencial", "target": "Dinâmica Molecular"},
    {"source": "Teoria dos Grafos", "target": "Dinâmica Molecular"},
    {"source": "Monte Carlo", "target": "Dinâmica Molecular", "relationship": "complementar"},
]

In [7]:
# Create the base chart
base = alt.Chart(
    alt.Data(
        values=nodes,
        format={"type": "json", "property": "nodes"},
    )
).encode(
    x=alt.X("x:Q", axis=None),
    y=alt.Y("y:Q", axis=None),
)

# Draw nodes
nodes = base.mark_circle().encode(
    color="group:N",
    tooltip=["id:N", "group:N"],
)

# Draw edges (workaround for older Altair without ConditionalPredicate)
edges = alt.Chart(
    alt.Data(
        values=edges,
        format={"type": "json", "property": "edges"},
    )
).transform_calculate(
    relationship="isValid(datum.relationship) ? datum.relationship : null"  # Use null instead of "none"
).mark_line(point=True).encode(
    x="x:Q",
    y="y:Q",
    detail="relationship:N", 
    color=alt.condition(
        alt.datum.relationship == "complementar",
        alt.value("blue"), 
        alt.value("grey"),
    ),
    tooltip=[
        alt.Tooltip("source:N", title="Source"),
        alt.Tooltip("target:N", title="Target"),
        alt.Tooltip(
            field="relationship:N",
            title="Relationship",
            # Condition applied directly to the field
            # (This may result in warning messages in older Altair versions, but should work)
            bin=alt.BinParams(maxbins=1),  # This forces Altair to treat the field as nominal 
        ),
    ],
).transform_filter(
    alt.datum.relationship != None  # Filter out null relationships here
)

SchemaValidationError: '1' is an invalid value for `maxbins`.

1 is less than the minimum of 2

In [5]:
# Combine nodes and edges
chart = (
    alt.layer(nodes, edges)
    .transform_lookup(
        lookup="id",
        from_=alt.LookupData(
            alt.Data(values=nodes, format={"type": "json", "property": "nodes"}),
            key="id",
            fields=["x", "y"],
        ),
    )
    .properties(width=800, height=400)
    .configure_scale(bandPaddingInner=0.8)
    .interactive()
)

# Save the chart
chart.save("molecular_dinamics_math_rel.json")

# Display the chart
chart

SchemaValidationError: `Tooltip` has no parameter named 'condition'

Existing parameter names are:
shorthand      bin         format       title   
aggregate      condition   formatType   type    
bandPosition   field       timeUnit             

See the help for `Tooltip` to read the full description of these parameters